# Tidy Data Exercises

In [312]:
import pandas as pd
import numpy as np
import os
import acquire
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# visualize
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(11, 9))
plt.rc('font', size=13)

# turn off pink warning boxes
import warnings
warnings.filterwarnings("ignore")

# acquire
from env import host, user, password
from pydataset import data

## 1. Attendance Data
### Load the attendance.csv file and calculate an attendnace percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [313]:
attendance_df = pd.read_csv('untidy-data/attendance.csv')

In [314]:
attendance_df.head()

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [315]:
attendance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4 non-null      object
 1   2018-01-01  4 non-null      object
 2   2018-01-02  4 non-null      object
 3   2018-01-03  4 non-null      object
 4   2018-01-04  4 non-null      object
 5   2018-01-05  4 non-null      object
 6   2018-01-06  4 non-null      object
 7   2018-01-07  4 non-null      object
 8   2018-01-08  4 non-null      object
dtypes: object(9)
memory usage: 416.0+ bytes


In [4]:
#rename column containing student names
attendance_df.rename(columns={'Unnamed: 0': 'student'}, inplace=True)

In [18]:
attendance_df

,student,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [19]:
#melt dates together with attendnace values
attendance_melt = attendance_df.melt(id_vars='student', var_name='date', value_name='attendance')

In [20]:
attendance_melt

,student,date,attendance
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T
5,Jane,2018-01-02,P
6,Billy,2018-01-02,T
7,John,2018-01-02,T
8,Sally,2018-01-03,T
9,Jane,2018-01-03,T


In [316]:
#create a function that takes in a letter grade and returns the corresponding numeric points value
def get_points(n):
    if n == 'P':
        return 2
    elif n == 'H':
        return 1
    elif n == 'A':
        return 0
    else:
        return 1.8
    
attendance_melt[['attendance_pts']] = attendance_melt['attendance'].apply(get_points)            

In [317]:
attendance_melt

,student,date,attendance,attendance_pts
0,Sally,2018-01-01,P,2.0
1,Jane,2018-01-01,A,0.0
2,Billy,2018-01-01,A,0.0
3,John,2018-01-01,P,2.0
4,Sally,2018-01-02,T,1.8
5,Jane,2018-01-02,P,2.0
6,Billy,2018-01-02,T,1.8
7,John,2018-01-02,T,1.8
8,Sally,2018-01-03,T,1.8
9,Jane,2018-01-03,T,1.8


In [103]:
#calculate the attendance grade by summing points per students divided by the total number of points 
#(2pts per day for 8 days)
attendance_melt.groupby('student').attendance_pts.sum()/16

student
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: attendance_pts, dtype: float64

### 2. Coffee Levels

- Read the coffee_levels.csv file.

In [318]:
coffee_df = pd.read_csv('untidy-data/coffee_levels.csv')

In [319]:
coffee_df

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891
9,17,x,0.391560


In [87]:
coffee_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   hour           30 non-null     int64  
 1   coffee_carafe  30 non-null     object 
 2   coffee_amount  30 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 848.0+ bytes


- Transform the data so that each carafe is in it's own column.

In [91]:
coffee_pivot = coffee_df.pivot_table(index = ['hour'], columns = 'coffee_carafe', values='coffee_amount')

In [92]:
coffee_pivot

coffee_carafe,x,y,z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947
13,0.310711,0.997464,0.398520
14,0.507288,0.058361,0.864464
15,0.215043,0.144644,0.436364
16,0.183891,0.544676,0.280621


- Is this the best shape for the data?

In [320]:
#I might prefer to tansform the data that each carafe is its own row and the columns are the hour.

coffee_pivot.T

hour,8,9,10,11,12,13,14,15,16,17
coffee_carafe,,,,,,,,,,
x,0.816164,0.451018,0.843279,0.335533,0.898291,0.310711,0.507288,0.215043,0.183891,0.391560
y,0.189297,0.521502,0.023163,0.235529,0.017009,0.997464,0.058361,0.144644,0.544676,0.594126
z,0.999264,0.915990,0.144928,0.311495,0.771947,0.398520,0.864464,0.436364,0.280621,0.436677


### Cake Recipes

- **a. Read the cake_recipes.csv data. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.**

In [321]:
cake_df = pd.read_csv('untidy-data/cake_recipes.csv')

In [322]:
cake_df.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


In [238]:
cake_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   recipe:position  8 non-null      object 
 1   225              8 non-null      float64
 2   250              8 non-null      float64
 3   275              8 non-null      float64
 4   300              8 non-null      float64
dtypes: float64(4), object(1)
memory usage: 448.0+ bytes


- **b. Tidy the data as necessary.**

In [324]:
#split column that has both recipe and oven position and expand to two separate columns
cake_df[['recipe', 'oven_position']] = cake_df['recipe:position'].str.split(':', expand = True)

In [325]:
#drop the original column that was split
cake_df.drop(columns={'recipe:position'}, inplace=True)

In [326]:
#melt oven temp and tastiness score
cake_melt = cake_df.melt(id_vars=['recipe','oven_position'], var_name='oven_temp', value_name='tasty_score')

In [242]:
cake_melt

,recipe,oven_position,oven_temp,tasty_score
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207
5,c,top,225,71.306308
6,d,bottom,225,52.799753
7,d,top,225,96.873178
8,a,bottom,250,53.912627
9,a,top,250,52.009735


- **c. Which recipe, on average, is the best? recipe b**

In [332]:
avg_tastiness_by_recipe = round(cake_melt.groupby('recipe').tasty_score.mean().max(),2)
best_recipe = cake_melt.groupby('recipe').tasty_score.mean().idxmax()
print(f"The best recipe based on average tastiness is recipe {best_recipe} with a tastiness score of {avg_tastiness_by_recipe}")

The best recipe based on average tastiness is recipe b with a tastiness score of 76.74


- **d. Which oven temperature, on average, produces the best results? 275**

In [294]:
avg_tastiness_by_oven_temps = cake_melt.groupby("oven_temp").tasty_score.mean().nlargest()

best_oven_temp = avg_oven_temps.idxmax()

In [296]:
print(f'{best_oven_temp} is the oven temp that produces the best results with a average tastiness score of {avg_tastiness_by_oven_temps[0].round(2)}')

275 is the oven temp that produces the best results with a average tastiness score of 74.89


- **e. Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees**

In [204]:
cake_melt[cake_melt.tasty_score == cake_melt.tasty_score.max()]

,recipe,oven_position,oven_temp,tasty_score
26,b,bottom,300,99.248541


#### Bonus: Tidy the gapminder1 dataset

In [297]:
gm1_df = pd.read_csv('untidy-data/gapminder1.csv')

In [298]:
gm1_df.head()

,year,country,measure,measurement
0,1955,Afghanistan,pop,8891209.0
1,1960,Afghanistan,pop,9829450.0
2,1965,Afghanistan,pop,10997885.0
3,1970,Afghanistan,pop,12430623.0
4,1975,Afghanistan,pop,14132019.0


In [299]:
gm1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2079 entries, 0 to 2078
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   year         2079 non-null   int64  
 1   country      2079 non-null   object 
 2   measure      2079 non-null   object 
 3   measurement  2079 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 65.1+ KB


In [300]:
gm1_df.tail()

,year,country,measure,measurement
2074,1985,Venezuela,fertility,3.6485
2075,1990,Venezuela,fertility,3.2500
2076,1995,Venezuela,fertility,2.9415
2077,2000,Venezuela,fertility,2.7230
2078,2005,Venezuela,fertility,2.5470


In [305]:
df_tidy = gm1_df.pivot_table(index=['country', 'year'], columns='measure', values='measurement')

In [306]:
df_tidy.reset_index(inplace=True)

In [311]:
df_tidy

measure,country,year,fertility,life_expect,pop
0,Afghanistan,1955,7.7000,30.332,8891209.0
1,Afghanistan,1960,7.7000,31.997,9829450.0
2,Afghanistan,1965,7.7000,34.020,10997885.0
3,Afghanistan,1970,7.7000,36.088,12430623.0
4,Afghanistan,1975,7.7000,38.438,14132019.0
...,...,...,...,...,...
688,Venezuela,1985,3.6485,70.190,16997509.0
689,Venezuela,1990,3.2500,71.150,19325222.0
690,Venezuela,1995,2.9415,72.146,21555902.0
691,Venezuela,2000,2.7230,72.766,23542649.0
